<a href="https://colab.research.google.com/github/deayalar/deeplearning_unitn/blob/main/DL_Project_wandb_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
!unzip -q dataset.zip -d dataset

--2021-06-24 17:26:40--  https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
Resolving market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)... 52.218.233.177
Connecting to market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)|52.218.233.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82925180 (79M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]  79.08M  41.8MB/s    in 1.9s    

2021-06-24 17:26:42 (41.8 MB/s) - ‘dataset.zip’ saved [82925180/82925180]



In [2]:
!rm -rf /content/deeplearning_unitn
!git clone https://github.com/deayalar/deeplearning_unitn.git

Cloning into 'deeplearning_unitn'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (239/239), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 239 (delta 131), reused 138 (delta 62), pack-reused 0
Receiving objects: 100% (239/239), 10.71 MiB | 5.56 MiB/s, done.
Resolving deltas: 100% (131/131), done.


In [7]:
!nvidia-smi

Thu Jun 24 17:59:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    27W /  70W |   1912MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
config = dict(
    wandb = False,
    device = "auto", # Select an specific device None to select automatically
    train_root = "/content/dataset/train",
    test_root = "/content/dataset/test", 
    queries_root = "/content/dataset/queries",
    attributes_file = "/content/dataset/annotations_train.csv",
    #train_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/train",
    #test_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/test",
    #queries_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/queries",
    #attributes_file = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/annotations_train.csv",
    dataset="Market1501",
    backbone = "resnet18",
    split = dict(
        full_training_size = 0.75,
        train_size = 0.8
    ),
    compose = dict(
        resize_h = 224,
        resize_w = 224
    ),
    epochs=1,
    training_batch_size=32,
    validation_batch_size=32,
    learning_rate=0.01,
    weight_decay=0.000001, 
    momentum=0.9,
    test_before_training=False,
    test_after_epochs=10,
    mAP_rank=15)

In [22]:
%cd /content/deeplearning_unitn

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm

import cost_functions
from evaluation import Evaluator
from datasets.reid_dataset import Market1501
from cost_functions import OverallLossWrapper
from utils.split_data import ValidationSplitter, TrainingSplitter
from models.reid_model import FinetunedModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

if config["device"] == "auto":
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
else:
    device = config["device"]
print(device)

/content/deeplearning_unitn
cuda:0


In [ ]:
!pip install wandb -q
import wandb
if config["wandb"]:
  wandb.login()

     |████████████████████████████████| 1.8MB 6.8MB/s 
     |████████████████████████████████| 174kB 22.4MB/s 
     |████████████████████████████████| 102kB 10.9MB/s 
     |████████████████████████████████| 133kB 26.7MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [47]:
def model_pipeline(hyperparameters):
    """
    This function corresponds to the basic pipeline of all tested models
    0) Split data
    1) Setup based on the configuration
    2) Train the model
    3) Test performance
    """
    config = hyperparameters
    if config["wandb"]:
      wandb.init(entity="dl_unitn", project="dl_project", config=hyperparameters)
      config = wandb.config
    print(config)
    
    train_set, val_set, val_queries = split_data(config)
    
    model, train_loader, val_loader, val_queries_loader, criterion, optimizer = setup(train_set, val_set, val_queries, config)
    id_ground_truth_dict = build_ground_truth(val_set, val_queries)

    print("Using "+ config["backbone"] + " as backbone")
    if config["test_before_training"]:
      test(model, val_loader, val_queries_loader, id_ground_truth_dict, config)

    train(model, train_loader, val_loader, criterion, optimizer, config)

    test(model, val_loader, val_queries_loader, id_ground_truth_dict, config, save_model=True)

    return model

In [48]:
def build_ground_truth(val_set, val_queries):
    values = []
    for q in val_queries:
        matches = []
        for idx_v, v in enumerate(val_set):
            if v.split("_")[0] == q.split("_")[0]:
                matches.append(idx_v)
        value = set(matches)
        values.append(value)
        
    ground_truth_dict = dict(zip(list(range(0, len(val_queries))), values))
    return ground_truth_dict


In [49]:
def split_data(config):
    """Returns a list with the names of theimages in each set"""
    splitter = ValidationSplitter(train_root=config["train_root"], 
                                  test_root=config["test_root"], 
                                  queries_root=config["queries_root"])
    train_set, val_set, val_queries = splitter.split(train_size=config["split"]["full_training_size"],
                                                     random_seed=42)
    return train_set, val_set, val_queries

def setup(train_set, val_set, val_queries, config):
    #Create pytorch Datasets
    train_composed = transforms.Compose([ transforms.Resize((config["compose"]["resize_h"], 
                                                      config["compose"]["resize_w"])),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                std=[0.229, 0.224, 0.225]),
                                          transforms.RandomErasing(p=0.6)])
    
    val_composed = transforms.Compose([transforms.Resize((config["compose"]["resize_h"], 
                                                      config["compose"]["resize_w"])),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                        std=[0.229, 0.224, 0.225])])
    
    train_dataset = Market1501(root_dir=config["train_root"],
                            attributes_file=config["attributes_file"],
                            images_list=train_set,
                            transform=train_composed)
                            
    val_dataset = Market1501(root_dir=config["train_root"],
                         attributes_file=config["attributes_file"],
                         images_list=val_set,
                         transform=val_composed)

    val_queries_dataset = Market1501(root_dir=config["train_root"],
                         attributes_file=config["attributes_file"],
                         images_list=val_queries,
                         transform=val_composed)

    train_loader = torch.utils.data.DataLoader(train_dataset, 
                                               batch_size=config["training_batch_size"], 
                                               shuffle=True, 
                                               num_workers=2)
                                               
    val_loader = torch.utils.data.DataLoader(val_dataset, 
                                             batch_size=config["validation_batch_size"], 
                                             shuffle=False, 
                                             num_workers=2)

    val_queries_loader = torch.utils.data.DataLoader(val_queries_dataset, 
                                             batch_size=config["validation_batch_size"],
                                             shuffle=False, 
                                             num_workers=2)

    attr_len = len(train_dataset[0][2]) #Number of attributes in the csv: 27
    print(f"Number of attributes: {attr_len}")
    model = FinetunedModel(architecture=config["backbone"] ,n_classes=attr_len).to(device)

    #This is a combination of the attributes classification loss and the triplet loss for identification
    criterion = OverallLossWrapper()
    optimizer = torch.optim.SGD(model.parameters(), 
                                lr=config["learning_rate"], 
                                weight_decay=config["weight_decay"], 
                                momentum=config["momentum"])
    
    return model, train_loader, val_loader, val_queries_loader, criterion, optimizer

In [50]:
def compute_centroids(model, loader):
  model.eval()
  print("Computing centroids")
  all_features = np.empty((0, model.feature_size))
  ids = []
  for batch_idx, (inputs, identity, attributes) in enumerate(tqdm(loader)):
    inputs = inputs.to(device)
    batch_features = model(inputs, get_features = True)
    all_features = np.concatenate((all_features, batch_features.cpu().detach().numpy()), axis=0)
    ids.extend(list(identity))

  ids = np.array([int(i) for i in ids])
  unique_ids = np.unique(ids)

  centroids = []
  for id in unique_ids:
    id_features = all_features[ids == id]
    centroid = np.mean(id_features, axis = 0)
    centroids.append(centroid)
  return centroids


def train(model, train_loader, val_loader, criterion, optimizer, config):
    print("Training...")
    print(train_loader)
    # tell wandb to watch what the model gets up to: gradients, weights, and more!
    if config["wandb"]:
         wandb.watch(model, criterion, log="all", log_freq=10)
    
    # Run training and track with wandb
    total_batches = len(train_loader) * config["epochs"]
    example_ct = 0  # number of seen examples
    batch_ct = 0

    centroids = compute_centroids(model, train_loader)

    model.train()
    for epoch in tqdm(range(config["epochs"])):
        for batch_idx, (inputs, identity, attributes) in enumerate(train_loader):
            loss = train_batch(inputs, identity, attributes, model, optimizer, criterion, centroids)

            example_ct +=  len(inputs)
            batch_ct += 1

            if ((batch_ct + 1) % 50) == 0:
                train_log(loss, example_ct, epoch)

        centroids = compute_centroids(model, train_loader)


def train_batch(inputs, identity, attributes, model, optimizer, criterion, centroids):
    inputs = inputs.to(device)
    attributes = attributes.to(device)
    
    # Forward pass
    # TODO: This could be improved in the architecture to return both at the same time and improve the training time
    output_attrs = model(inputs)
    output_features = model(inputs, get_features=True)

    # Apply the loss
    loss = criterion(output_attrs, attributes, output_features, identity, centroids)
    
    optimizer.zero_grad()
    # Backward pass
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [51]:
def train_log(loss, example_ct, epoch):
    loss = float(loss)
    if config["wandb"]:
        wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Epoch {epoch}: Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [52]:
def get_features_from_loader(model, loader):
    model.eval()
    with torch.no_grad():
        all_features = torch.zeros(len(loader.dataset), model.feature_size)
        for batch_idx, (inputs, ids, attr) in enumerate(tqdm(loader)):
                inputs = inputs.to(device)
                features = model(inputs, get_features=True)
                for in_batch, f in enumerate(features):
                    all_features[(batch_idx * loader.batch_size) + in_batch] = f
        return all_features


def test_mAP(model, gallery_loader, queries_loader, ground_truth_dict, config, save_model=False):
    """
    This function returns the mAP performance of a given model 
    Params:
    model: model to be evaluated
    gallery: tensor that contains the feature representations of the target images in validation or test set
    queries: tensor that contains feature representations of the queries
    rank: top number of elements to retrieve

    Returns:
    mAP performance of the model
    """

    # Run the model on some test examples
    with torch.no_grad():
        
        # Compute the features for queries and gallery
        print("Computing gallery features...")
        gallery_features = get_features_from_loader(model, gallery_loader)
        print("Computing query features...")
        query_features = get_features_from_loader(model, queries_loader)
        
        # Build the cosine similarity matrix between the all the queries and all the elements in gallery
        print("Computing cosine similarities...")
        sims_matrix = torch.empty(query_features.size()[0], gallery_features.size()[0])
        for idx, q in enumerate(query_features):
            sims_matrix[idx] = F.cosine_similarity(q, gallery_features, dim=-1)
        
        print("Similarity matrix shape: " + str(sims_matrix.size()))
        sorted_index = torch.argsort(sims_matrix, dim=1, descending=True)
        top_k = sorted_index.narrow_copy(dim=1, start=0, length=config["mAP_rank"])

        #Build the dictionary to compute the mAP
        predictions_dict = {idx:  r for idx, r in enumerate(top_k.tolist())}
        mAP = Evaluator.evaluate_map(predictions_dict, ground_truth_dict)
        
        print(f"mAP: {mAP}")
        if config["wandb"]:
            wandb.log({"mAP": mAP})

In [53]:
def get_attributes_from_loader(model, loader):
    model.eval()

    all_predictions = np.empty(shape=[0, 27], dtype=np.byte)
    all_attrs = np.empty(shape=[0, 27], dtype=np.byte)

    with torch.no_grad():
        for batch_idx, (inputs, ids, attr) in enumerate(tqdm(loader)):
                inputs = inputs.to(device)
                outputs = model(inputs, get_features=False)
                #print("attr:",attr)
                predictions = torch.empty(attr.size()[1], attr.size()[0])
                for attr_idx, output in enumerate(outputs):
                    if output.size()[1] == 1: #If the output is binary
                        pred = torch.round(torch.squeeze(output, 1))
                    else: #Otherwise it is multiclass
                        pred = torch.argmax(output, dim=1)
                    predictions[attr_idx] = pred

                predictions = torch.transpose(predictions, 0, 1).cpu().numpy()
                attr = attr.cpu().numpy()

                all_predictions = np.append(all_predictions, predictions, axis=0)
                #print("all_predictions shape: ", all_predictions.shape)
                all_attrs = np.append(all_attrs, attr, axis=0)
                #print("all_attrs shape: ", all_attrs.shape)
        return all_predictions, all_attrs

def test_attributes(model, loader, config):
    print("Computing attributes...")
    predictions, attr = get_attributes_from_loader(model, loader)
    print("pred shape: ", predictions.shape)
    print("attr shape: ", attr.shape)

    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []

    for i in range(0, predictions.shape[1]):
        y_true, y_pred = attr[:, i], predictions[:, i]
        accuracy_list.append(accuracy_score(y_true, y_pred))
        if i == 0: #If it is age
            precision_list.append(precision_score(y_true, y_pred, average='macro'))
            recall_list.append(recall_score(y_true, y_pred, average='macro'))
            f1_score_list.append(f1_score(y_true, y_pred, average='macro'))
        else:
            precision_list.append(precision_score(y_true, y_pred))
            recall_list.append(recall_score(y_true, y_pred))
            f1_score_list.append(f1_score(y_true, y_pred))

    average_acc = np.mean(accuracy_list)
    average_precision = np.mean(precision_list)
    average_recall = np.mean(recall_list)
    average_f1score = np.mean(f1_score_list)

    print("accuracy_list: ", accuracy_list)
    print("precision_list: ", precision_list)
    print("recall_list: ", recall_list)
    print("f1_score_list: ", f1_score_list)

    print("average_acc: ", average_acc)
    print("average_precision: ", average_precision)
    print("average_recall: ", average_recall)
    print("average_f1score: ", average_f1score)

    if config["wandb"]:
            wandb.log({"accuracy_list": accuracy_list})
            wandb.log({"precision_list": precision_list})
            wandb.log({"recall_list": recall_list})
            wandb.log({"f1_score_list": f1_score_list})

            wandb.log({"average accuracy": average_acc})
            wandb.log({"average precision": average_precision})
            wandb.log({"average recall": average_recall})
            wandb.log({"average f1": average_f1score})

In [54]:
def test(model, gallery_loader, queries_loader, ground_truth_dict, config, save_model=False):
    print("Testing")
    model.eval()

    test_mAP(model, gallery_loader, queries_loader, ground_truth_dict, config)
    test_attributes(model, gallery_loader, config)

    if save_model :
      # Save the model in the exchangeable ONNX format
      inputs, id, attr = next(iter(gallery_loader))
      torch.onnx.export(model, inputs.to(device), "model.onnx", True)
      if config["wandb"]:
        wandb.save("model.onnx")

In [55]:
model = model_pipeline(config)

{'wandb': False, 'device': 'auto', 'train_root': '/content/dataset/train', 'test_root': '/content/dataset/test', 'queries_root': '/content/dataset/queries', 'attributes_file': '/content/dataset/annotations_train.csv', 'dataset': 'Market1501', 'backbone': 'resnet18', 'split': {'full_training_size': 0.75, 'train_size': 0.8}, 'compose': {'resize_h': 224, 'resize_w': 224}, 'epochs': 1, 'training_batch_size': 32, 'validation_batch_size': 32, 'learning_rate': 0.01, 'weight_decay': 1e-06, 'momentum': 0.9, 'test_before_training': False, 'test_after_epochs': 10, 'mAP_rank': 15}
Extract queries proportion: 0.11
Identities in train set: 563
Identities in validation set: 188
Train set size: 9646
Validation set size: 2975
Number of validation queries: 368
Number of attributes: 27
Backbone feature size: 512
Using resnet18 as backbone
Training...
Computing centroids


/content/deeplearning_unitn/cost_functions/__init__.py:62: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1025.)
  dist.addmm_(1, -2, inputs, inputs.t())


Epoch 0: Loss after 01568 examples: 9.254
Epoch 0: Loss after 03168 examples: 8.913
Epoch 0: Loss after 04768 examples: 8.869
Epoch 0: Loss after 06368 examples: 6.667
Epoch 0: Loss after 07968 examples: 8.096
Epoch 0: Loss after 09568 examples: 8.392
Computing centroids




Testing
Computing gallery features...



Computing query features...



Computing cosine similarities...
Similarity matrix shape: torch.Size([368, 2975])
mAP: 0.3560101308852766
Computing attributes...



pred shape:  (2975, 27)
attr shape:  (2975, 27)
accuracy_list:  [0.8507563025210084, 0.7647058823529411, 0.718655462184874, 0.9324369747899159, 0.8789915966386554, 0.866218487394958, 0.9334453781512605, 0.8265546218487395, 0.9761344537815126, 0.8272268907563025, 0.9314285714285714, 0.8709243697478991, 0.9300840336134454, 0.971764705882353, 0.973109243697479, 0.8988235294117647, 0.935798319327731, 0.9559663865546218, 0.838655462184874, 0.9475630252100841, 0.9899159663865547, 1.0, 0.9932773109243698, 0.8114285714285714, 0.838655462184874, 0.987563025210084, 0.9284033613445378]
precision_list:  [0.2126890756302521, 0.48498845265588914, 0.4089068825910931, 0.0, 0.8817533129459735, 0.8555066079295154, 0.9334453781512605, 0.7425944841675178, 0.0, 0.8391489361702128, 0.7693798449612403, 0.7276536312849162, 0.6067708333333334, 0.7954545454545454, 0.8309859154929577, 0.48484848484848486, 0.7403100775193798, 0.7192982456140351, 0.8770562770562771, 0.6190476190476191, 0.8777777777777778, 0.0, 0.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and

In [56]:
train_set, val_set, val_queries = split_data(config)
model, train_loader, val_loader, val_queries_loader, criterion, optimizer = setup(train_set, val_set, val_queries, config)
#id_ground_truth_dict = build_ground_truth(val_set, val_queries)

def compute_centroids(model, train_loader):
  #all_features = torch.empty(0, model.feature_size)
  all_features = np.empty((0, model.feature_size))
  ids = []
  for batch_idx, (inputs, identity, attributes) in enumerate(tqdm(train_loader)):
    inputs = inputs.to(device)
    batch_features = model(inputs, get_features = True)
    all_features = np.concatenate((all_features, batch_features.cpu().detach().numpy()), axis=0)
    ids.extend(list(identity))
  return all_features, np.array([int(i) for i in ids])

features, ids = compute_centroids(model, train_loader)
print(features.shape)
print(len(ids))


Extract queries proportion: 0.11
Identities in train set: 563
Identities in validation set: 188
Train set size: 9646
Validation set size: 2975
Number of validation queries: 368
Number of attributes: 27
Backbone feature size: 512



(9646, 512)
9646


In [57]:
print(len(ids))
unique_ids = np.unique(ids)
print(ids)
centroids = []
for id in unique_ids:
  id_features = features[ids == id]
  centroid = np.mean(id_features, axis = 0)
  centroids.append(centroid)

# tid = "0005"
# pos = np.flatnonzero(unique_ids==int(tid))[0]
# print(pos)



9646
[1064  917   30 ... 1444  906  305]
